[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). All rights reserved.

# DIY Covid-19 Dashboard

The following Dashboard can be used to compare COVID figures for the London Borough of Lewisham and England on the whole.

These figures specifically are;
    
- The number of daily cases reported.
- The cumulative total of residents who have received their first vaccination.
- The cumulative total of residents who have received their second vaccination.

It should be noted that data for England begins from January 2020, whereas data for Lewisham was gathered starting April 2020.

In [3]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [ ]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [ ]:
def parse_date(datestring):
    return pd.to_datetime(datestring, format="%Y-%m-%d") # convert a date string into a pandas datetime object

def wrangle_data_file(file): # wrangling the data for the json file, for when the dashboard first loads
    with open(file, "rt") as INFILE:
        data=json.load(INFILE)
    datalist=data['data'] # grabbing the list of dicitionaries within the json file
    dates=[dictionary["Date"] for dictionary in datalist]
    dates.sort()
    # extracted dates and sorted
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1]) # converting into pandas date objects as above, and sorting from earliest to latest
    index=pd.date_range(startdate, enddate, freq='D') # creating date range index for frame
    df=pd.DataFrame(index=index, columns=['New Cases', 'First Doses', 'Second Doses']) # defining frame
    for entry in datalist:
        date=parse_date(entry['Date'])
        for column in ['New Cases', 'First Doses', 'Second Doses']:
            if pd.isna(df.loc[date, column]): 
                value= float(entry[column]) if entry[column]!=None else 0.0
                df.loc[date, column]=value # filling the frame, replacing nones with zeroes
            
    df.fillna(0.0, inplace=True) # filling any other gaps

    return df


def wrangle_data_raw(rawdata): # wrangling the raw data, for when we refresh from the API
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    # loading data from the json file
    data=rawdata
    datalist=data['data'] # grabbing the list of dicitionaries within the json file
    dates=[dictionary["Date"] for dictionary in datalist]
    dates.sort()
    # extracted dates and sorted
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1]) # converting into pandas date objects as above, and sorting from earliest to latest
    index=pd.date_range(startdate, enddate, freq='D') # creating date range index for frame
    df=pd.DataFrame(index=index, columns=['New Cases', 'First Doses', 'Second Doses']) # defining frame
    for entry in datalist:
        date=parse_date(entry['Date'])
        for column in ['New Cases', 'First Doses', 'Second Doses']:
            if pd.isna(df.loc[date, column]): 
                value= float(entry[column]) if entry[column]!=None else 0.0
                df.loc[date, column]=value # filling the frame, replacing nones with zeroes
            
    df.fillna(0.0, inplace=True) # filling any other gaps

    return df

lewishamdf=wrangle_data_file("lewishamvac.json") # lewisham dataframe
englanddf=wrangle_data_file("englandvac.json") # england dataframe

In [ ]:
def access_api_lew(): # data populating the Lewisham graph (and the json)
    filtersbtn=[
    'areaType=utla',
    'areaName=Lewisham'] # looking at data from the London Borough of Lewisham
    structurebtn={
    "Date": "date",
    "New Cases": "newCasesByPublishDate",
    "First Doses": "cumPeopleVaccinatedFirstDoseByVaccinationDate",
    "Second Doses": "cumPeopleVaccinatedSecondDoseByVaccinationDate"} # daily cases vs vaccine uptake
    api = Cov19API(filters=filtersbtn, structure=structurebtn) # defining api as the covid api, "sliced" with above filters and structures
    lew_btn=api.get_json() # getting the data and storing it in variable
    return lew_btn

def access_api_eng(): # doing the same as above but with England data, to compare the Borough to the country on the whole
    filtersbtn=[
    'areaType=nation',
    'areaName=England']
    structurebtn={
    "Date": "date",
    "New Cases": "newCasesByPublishDate",
    "First Doses": "cumPeopleVaccinatedFirstDoseByVaccinationDate",
    "Second Doses": "cumPeopleVaccinatedSecondDoseByVaccinationDate"}
    api = Cov19API(filters=filtersbtn, structure=structurebtn)
    eng_btn=api.get_json()
    return eng_btn

In [ ]:
def api_button_callback_lew(button):
    apidata=access_api_lew() # get the data using the function above
    global lewishamdf
    lewishamdf=wrangle_data_raw(apidata) # wrangle the data, using global to overwrite the variable containing the "canned" dataframe
    refresh_graph_lew() # forcing a redraw of the graph using function below, which means the dataframe updates with fresh data
    apibutton_lew.icon="check" # change icon
    apibutton_lew.disabled=False # have elected to keep the button available, just in case...

    
apibutton_lew=wdg.Button(
    description='Refresh Lewisham Data',
    disabled=False,
    button_style='info',
    tooltip="Click here to refresh data for Lewisham",
    icon='refresh'
)


def api_button_callback_eng(button):
    # as above, but for england data
    apidata=access_api_eng()
    global englanddf
    englanddf=wrangle_data_raw(apidata)
    refresh_graph_eng()
    apibutton_eng.icon="check"
    apibutton_eng.disabled=False

    
apibutton_eng=wdg.Button(
    description='Refresh England Data',
    disabled=False,
    button_style='info',
    tooltip="Click here to refresh data for England",
    icon='refresh'
)

apibutton_lew.on_click(api_button_callback_lew) # calls the above functions on click of the buttons
apibutton_eng.on_click(api_button_callback_eng) 



## Lewisham Data

The below graph shows the number of Daily Cases reported, while also tracking the number of residents as they become vaccinated.

In [ ]:
lewcols=wdg.SelectMultiple(
    options=['New Cases', 'First Doses', 'Second Doses'], # options available
    value=['New Cases', 'First Doses', 'Second Doses'], # values
    rows=3, # rows of the selection box
    description='Filters',
    disabled=False
)

lewscale=wdg.RadioButtons(

    
    options=['Linear', 'Logarithmic'], # options for choosing linear or log
    description='Scale:',
    disabled=False
)

controls=wdg.HBox([lewcols, lewscale])

def lew_graph(graphcolumns,graphscale):
    if graphscale=='Linear':
        logscale=False
    else:
        logscale=True # enabling log if linear isn't selected
    ncols=len(graphcolumns)
    if ncols>0: # if there's columns...
        lewishamdf[list(graphcolumns)].plot(logy=logscale) # ...plot the graph, using the dataframe columns in a list
        plt.show() # and show it!
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")  # if no column is selected, print a message instead
    
output=wdg.interactive_output(lew_graph, {'graphcolumns': lewcols, "graphscale":lewscale}) # put it in a widget, calling the lewisham graph using the columns and scale provied above



def refresh_graph_lew():
    current=lewscale.value
    if current==lewscale.options[0]: # if value and options are in the same index in the list
        other=lewscale.options[1] # other is at a different point
    else:
        other=lewscale.options[0] # if not, other is still a different point
    lewscale.value=other # change the value to a different one
    lewscale.value=current # change it back to what it was
    

form=wdg.HBox([output,controls]) # put it in a horizontal widget, graph and controls next to each other


display(form) 
display(apibutton_lew)

## England Data

This section shows the same data as above, but on a national scale.

In [ ]:
# all as above, but for england graph
engcols=wdg.SelectMultiple(
    options=['New Cases', 'First Doses', 'Second Doses'],
    value=['New Cases', 'First Doses', 'Second Doses'],
    rows=3,
    description='Filters',
    disabled=False
)

engscale=wdg.RadioButtons(

    
    options=['Linear', 'Logarithmic'],
    description='Scale:',
    disabled=False
)

controls=wdg.HBox([engcols, engscale])

def eng_graph(graphcolumns,graphscale):
    if graphscale=='Linear':
        logscale=False
    else:
        logscale=True
    ncols=len(graphcolumns)
    if ncols>0:
        englanddf[list(graphcolumns)].plot(logy=logscale)
        plt.show()
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")


output=wdg.interactive_output(eng_graph, {'graphcolumns': engcols, "graphscale":engscale})



def refresh_graph_eng():
    current=engscale.value
    if current==engscale.options[0]:
        other=engscale.options[1]
    else:
        other=engscale.options[0]
    engscale.value=other
    engscale.value=current


form=wdg.HBox([output,controls])


display(form) 
display(apibutton_eng)

The local data for Lewisham is more or less on par with the data for the country.

**(C) 2021 Jordan O'Shea** [ec21933@qmul.ac.uk](mailto:ec21933@qmul.ac.uk), all rights reserved. *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*